In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PowerTransformer, StandardScaler, FunctionTransformer, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor, LGBMClassifier
import plotly.express as px
from plotly.subplots import make_subplots
from numba import jit

In [30]:
# Чтение/переименование

def read_rename(filename):
    df = pd.read_csv(filename,index_col='Unnamed: 0')
    df.rename(columns={'SeriousDlqin2yrs':'Credit delay', 'age':'Age', 
                         'RevolvingUtilizationOfUnsecuredLines':'BalanceCreditRate', 
                         'NumberOfTime30-59DaysPastDueNotWorse':'Delay 30-59', 'DebtRatio':'IncomeDebtRatio', 
                         'MonthlyIncome':'Income', 'NumberOfOpenCreditLinesAndLoans':'NumberOpenCredit', 
                         'NumberOfTimes90DaysLate':'Delay >90',
                         'NumberOfTime60-89DaysPastDueNotWorse':'Delay 60-89',
                         'NumberRealEstateLoansOrLines':'IpotekaNum', 'NumberOfDependents':'Kids'},
               inplace=True)
    return df

In [31]:
df_train = read_rename('cs-training.csv')

In [32]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
Credit delay         150000 non-null int64
BalanceCreditRate    150000 non-null float64
Age                  150000 non-null int64
Delay 30-59          150000 non-null int64
IncomeDebtRatio      150000 non-null float64
Income               120269 non-null float64
NumberOpenCredit     150000 non-null int64
Delay >90            150000 non-null int64
IpotekaNum           150000 non-null int64
Delay 60-89          150000 non-null int64
Kids                 146076 non-null float64
dtypes: float64(4), int64(7)
memory usage: 13.7 MB


#### Оценим распределение данных.

In [33]:
df_train.describe()

,Credit delay,BalanceCreditRate,Age,Delay 30-59,IncomeDebtRatio,Income,NumberOpenCredit,Delay >90,IpotekaNum,Delay 60-89,Kids
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


#### Оценим распределения значений переменных.
К сожалению, от plotly на больших выборках все виснет, поэтому приведу скрины распределений переменных.
<div style='display:flex flex-wrap:wrap flex-direction: row'>
    <img src='img/3059.png' style="width:75%">
    <img src='img/6089.png' style="width:75%">
    <img src='img/age.png' style="width:75%">
    <img src='img/balance-credit.png' style="width:75%">
    <img src='img/countipotek.png' style="width:75%">
    <img src='img/income.png' style="width:75%">
    <img src='img/income-cost.png' style="width:75%">
    <img src='img/kids.png' style="width:75%">
    <img src='img/open credit.png' style="width:75%">
    <img src='img/over90.png' style="width:75%">
</div>

In [36]:
# Конвейер
conveyor = Pipeline([('imputation', SimpleImputer(strategy='median')),
                    ('powertrans', StandardScaler()),
                    ('classifier', 
                     LogisticRegression(penalty='l2', solver='lbfgs', C=1, random_state=0, max_iter=2000))])

# Кросс-валидация 
cv_avg = cross_val_score(conveyor, 
                         df_train.drop(columns='Credit delay'), 
                         df_train['Credit delay'],
                         scoring='roc_auc', 
                         cv=3).mean()

print('Средняя AUC ROC на обучении по перекрестной проверке: {:.3f}'.format(cv_avg))

Средняя AUC ROC на обучении по перекрестной проверке: 0.698


In [38]:
# Конвейер
conveyor = Pipeline([('imputation', SimpleImputer()),
                    ('powertrans', PowerTransformer()),
                    ('classifier', LogisticRegression(penalty='l2', solver='lbfgs', C=1, random_state=0, max_iter=2000))])

# Кросс-валидация 
cv_avg = cross_val_score(conveyor, 
                         df_train.drop(columns='Credit delay'), 
                         df_train['Credit delay'],
                         scoring='roc_auc', 
                         cv=3).mean()

print('Средняя AUC ROC на обучении по перекрестной проверке: {:.3f}'.format(cv_avg))

Средняя AUC ROC на обучении по перекрестной проверке: 0.851


`PowerTransformer()` из коробки дал значительный прирост - `+0.153` к AUC ROC по сравнению с простой стандартизацией.

Попробуем более детальный подход к преобразованию данных, уточним его для каждого отдельного столбца. В зависимости от коэффициента ассиметрии применим адекватный по силе метод преобразования. 
Но прежде, вычислим коеффициенты ассиметрии для каждого предиктора.

In [39]:
df_train.skew(axis=0, skipna=True)

Credit delay           3.468857
BalanceCreditRate     97.631574
Age                    0.188995
Delay 30-59           22.597108
IncomeDebtRatio       95.157793
Income               114.040318
NumberOpenCredit       1.215314
Delay >90             23.087345
IpotekaNum             3.482484
Delay 60-89           23.331743
Kids                   1.588242
dtype: float64

Выделяются четыре группы:
<ul>
    <li>"Возраст" - самая "нормальная" переменная, слегка скошена вправо.<br>Для нее оставим преобразование Йео-Джонсона, <br>так как она не обладает большим потенциалом улучшения</li>
    <li>"Иждивенцы", "Число ипотек", "Число открытых кредитов" явно скошены вправо, но хвосты не очень тяжелые. <br>Эти переменные преобразуем извлечением квадратного корня.</li>
    <li>"Просрочки 30-50", "Просрочки 60-89", "Просрочки >90" сильно скошены вправо и хвосты значительны по масштабам. <br>Эти переменные преобразуем логарифмированием.</li>
    <li>"Доход" и "Отношение доходы-расходы" значительно скошены вправо, хвосты удаляются от медиан на много порядков. <br>Для этих предикторов выполним преобразование обращением</li>
</ul>
    

In [40]:
df_train.columns

Index(['Credit delay', 'BalanceCreditRate', 'Age', 'Delay 30-59',
       'IncomeDebtRatio', 'Income', 'NumberOpenCredit', 'Delay >90',
       'IpotekaNum', 'Delay 60-89', 'Kids'],
      dtype='object')

In [45]:
# Создадим списки признаков по степени их ассиметрии
semi_normal_column = ['Age']
light_skewed_columns = ['Kids', 'IpotekaNum', 'NumberOpenCredit']
medium_skewed_columns = ['Delay 30-59', 'Delay 30-59', 'Delay >90']
hard_skewed_columns = ['Income', 'IncomeDebtRatio']

# Определим функции преобразования и ускорим их Numb'ой
@jit
def square_root(x):
    return np.sqrt(x)


@jit
def ln(x):
    return np.log(x+1)


@jit
def root_4th_power(x):
    return 1/(x+0.1)


# Создадим конвейеры для соответствующих степеней ассиметрии
semi_normal_pipe = Pipeline([('imputation', SimpleImputer(strategy='mean')),
                             ('powertrans', PowerTransformer()),
                             ('k-bins', KBinsDiscretizer(n_bins=5,strategy='kmeans', encode='onehot-dense'))])

light_skewed_pipe = Pipeline([('imputation', SimpleImputer(strategy='median')),
                    ('light_transform', FunctionTransformer(func=square_root, validate=True))])

medium_skewed_pipe = Pipeline([('imputation', SimpleImputer(strategy='median')),
                    ('medium_transform', FunctionTransformer(func=ln, validate=True))])

hard_skewed_pipe = Pipeline([('imputation', SimpleImputer(strategy='median')),
                    ('hard_transform', FunctionTransformer(func=root_4th_power, validate=True))])

# Соберем все вместе в список перед подачей в ColumnTransformer()
transformation_plan = [('semi-normal', semi_normal_pipe, semi_normal_column),
                       ('light', light_skewed_pipe, light_skewed_columns),
                       ('medium', medium_skewed_pipe, medium_skewed_columns),
                       ('hard', hard_skewed_pipe, hard_skewed_columns)]

transfromer = ColumnTransformer(transformers=transformation_plan)

ml_pipe = Pipeline([('trafo', transfromer), ('lr', LogisticRegression(penalty='l2', 
                                                                      solver='lbfgs', 
                                                                      C=1, 
                                                                      random_state=0, 
                                                                      max_iter=2000))])

# Кросс-валидация 
cv_avg = cross_val_score(ml_pipe, 
                         df_train.drop(columns='Credit delay'), 
                         df_train['Credit delay'],
                         scoring='roc_auc', 
                         cv=3).mean()

print('Средняя AUC ROC на обучении по перекрестной проверке: {:.3f}'.format(cv_avg))

Средняя AUC ROC на обучении по перекрестной проверке: 0.800


"Персональная" обработка данных по степени ассиметрии распределения не помогла, качество относительно чистого `PowerTransformer`'a упала. Проверим данные на мультиколлинеарность. Посчитаем VIF для предикторов

In [47]:
df_vif = df_train.drop(columns='Credit delay').corr()
vifs = pd.Series(np.linalg.inv(df_vif.values).diagonal(), index=df_vif.index)
print(vifs)

BalanceCreditRate     1.000364
Age                   1.084436
Delay 30-59          40.874960
IncomeDebtRatio       1.019786
Income                1.023441
NumberOpenCredit      1.284064
Delay >90            73.099495
IpotekaNum            1.275401
Delay 60-89          91.342035
Kids                  1.076849
dtype: float64


Логично, что просрочки на разные периоды во много объясняют одну и ту же изменчивость в наборе данных. 
Удалим переменную с наибольшим фактором инфляции дисперсии - Просрочки более 90 дней

In [48]:
df_vif = df_train.drop(columns=['Credit delay', 'Delay >90']).corr()
vifs = pd.Series(np.linalg.inv(df_vif.values).diagonal(), index=df_vif.index)
print(vifs)

BalanceCreditRate     1.000364
Age                   1.084232
Delay 30-59          39.231900
IncomeDebtRatio       1.019784
Income                1.023422
NumberOpenCredit      1.276255
IpotekaNum            1.274947
Delay 60-89          39.284704
Kids                  1.076848
dtype: float64


Удалим вторую переменную - просрочки 60-89 дней

In [49]:
df_vif = df_train.drop(columns=['Credit delay', 'Delay >90', 'Delay 60-89']).corr()
vifs = pd.Series(np.linalg.inv(df_vif.values).diagonal(), index=df_vif.index)
print(vifs)

BalanceCreditRate    1.000363
Age                  1.082718
Delay 30-59          1.006395
IncomeDebtRatio      1.019779
Income               1.023395
NumberOpenCredit     1.265936
IpotekaNum           1.274776
Kids                 1.075571
dtype: float64


Хотя в подмножестве переменных с просрочкой 30-59 дней уже нет мультиколлинеарности, проверим сделаем еще один эксперимент - уберем просрочки от 30 до 89 дней, а оставим просрочки свыше 90 дней. Мотивация - заемщики с тяжкими просрочками все-таки более склонны к дефолту.

In [50]:
df_vif = df_train.drop(columns=['Credit delay', 'Delay 30-59','Delay 60-89']).corr()
vifs = pd.Series(np.linalg.inv(df_vif.values).diagonal(), index=df_vif.index)
print(vifs)

BalanceCreditRate    1.000363
Age                  1.082203
IncomeDebtRatio      1.019781
Income               1.023394
NumberOpenCredit     1.268458
Delay >90            1.009390
IpotekaNum           1.274862
Kids                 1.075696
dtype: float64


Получили эквивалентный набор признаков в контексте мультиколлинеарности. Будем работать с ним. Воспользуемся первым универсальным конвейером.

In [52]:
# Конвейер
conveyor = Pipeline([('imputation', SimpleImputer(strategy='median')),
                    ('powertrans', PowerTransformer()),
                    ('classifier', LogisticRegression(penalty='l2', 
                                                      solver='lbfgs', 
                                                      C=1, 
                                                      random_state=0, 
                                                      max_iter=2000))])

# Кросс-валидация 
cv_avg = cross_val_score(conveyor, 
                         df_train.drop(columns=['Credit delay', 'Delay 30-59','Delay 60-89']), 
                         df_train['Credit delay'],
                         scoring='roc_auc', 
                         cv=3).mean()

print('Средняя AUC ROC на обучении по перекрестной проверке: {:.3f}'.format(cv_avg))

Средняя AUC ROC на обучении по перекрестной проверке: 0.824


Качество упало.  
**Вопрос: Выходит, формальная мультиколлинеарность не всегда вредит данным?**  
Актуальный результат для регрессии:  
<p style='font-family: Courier New'>Средняя AUC ROC на обучении по перекрестной проверке: 0.851</p>


## GBDT + Scopt

Проверим LigthGBM классификатор из коробки

In [53]:
from sklearn.model_selection import train_test_split

train, test, y_train, y_split = train_test_split(df_train.drop(columns=['Credit delay']), 
                                                 df_train['Credit delay'], test_size=0.3,random_state=0)

In [55]:
si = SimpleImputer(strategy='median')

si.fit(train)
test = si.transform(test)


In [56]:
# Конвейер
conveyor = Pipeline([('imputation', SimpleImputer(strategy='median')),
                    ('classifier', LGBMClassifier(random_state=0)),
                    ])

# Кросс-валидация 
cv_avg = cross_val_score(conveyor, 
                         df_train.drop(columns=['Credit delay']), 
                         df_train['Credit delay'],
                         scoring='roc_auc', 
                         cv=3).mean()

print('Средняя AUC ROC на обучении по перекрестной проверке: {:.3f}'.format(cv_avg))

Средняя AUC ROC на обучении по перекрестной проверке: 0.863


Сразу лучше логистической регрессии на `0.012`. Теперь попробуем настроить LightGBMClassifier силами байесовской оптимизации

In [57]:
from skopt import BayesSearchCV, gp_minimize
from skopt.space import Real, Integer

In [71]:
# Пространство значений гиперпараметров

param_space = {"n_estimators":Integer(100, 1000),
               "learning_rate":Real(0.0001, 0.1, prior='log-uniform'),
               "max_depth":Integer(1,10),
               "subsample":Real(0.1,1, prior='uniform'),
               "reg_alpha":Real(0.0001, 1, prior='log-uniform'),
              "reg_lambda":Real(0.0001, 1, prior='log-uniform')}

In [72]:
bayopt = BayesSearchCV(LGBMClassifier(n_jobs=-1), param_space, n_iter=100, random_state=0, cv=3)

In [73]:
bayopt.fit(df_train.drop(columns=['Credit delay']), df_train['Credit delay'])

D:\miniconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning:

The objective has been evaluated at this point before.

D:\miniconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning:

The objective has been evaluated at this point before.

D:\miniconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning:

The objective has been evaluated at this point before.

D:\miniconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning:

The objective has been evaluated at this point before.

D:\miniconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning:

The objective has been evaluated at this point before.

D:\miniconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning:

The objective has been evaluated at this point before.

D:\miniconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning:

The objective has been evaluated at this point before.

D:\miniconda\lib\site-packages\skopt\optimizer\optimize

BayesSearchCV(cv=3, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_iter=100, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=0,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'n_estimators': Integer(low=100, high=1000), 'learning_rate': Real(low=0.0001, high=0.1, prior='log-uniform', transform='identity'), 'max_depth': Integer(low=1, high=10), 'subsample': Real(low=0.1, high=1, prior='uniform', transform='identity'), 'reg_alpha': Real(low=0.0001, high=1, prior='log-un

In [76]:
# Результаты
bayopt.best_estimator_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.08435967022181244,
        max_depth=10, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=124, n_jobs=-1, num_leaves=31,
        objective=None, random_state=None, reg_alpha=0.9689337888861052,
        reg_lambda=0.0007342423179432522, silent=True,
        subsample=0.9529615658111046, subsample_for_bin=200000,
        subsample_freq=0)

In [77]:
# Применим лучшую модель в конвейере

# Конвейер
conveyor = Pipeline([('imputation', SimpleImputer(strategy='median')),
                    ('classifier', bayopt.best_estimator_),
                    ])

# Кросс-валидация 
cv_avg = cross_val_score(conveyor, 
                         df_train.drop(columns=['Credit delay']), 
                         df_train['Credit delay'],
                         scoring='roc_auc', 
                         cv=3).mean()

print('Средняя AUC ROC на обучении по перекрестной проверке: {:.3f}'.format(cv_avg))

Средняя AUC ROC на обучении по перекрестной проверке: 0.864


Незначительный прирост